# Goal

Here, we want to determine the best model parameters to maximize inference and generalizability of the model across human and mouse transcriptional data.

# Import

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import decoupler as dc
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, RidgeCV
from joblib import Parallel, delayed

import sys
sys.path.insert(0, '/data1/rudenska/EYW/git_projects/SIG13/functions')
import scanpy_custom as scc

%load_ext autoreload
%autoreload 2

# Prepare Calibration Data

## SIG14 Import

Bulk RNA-seq of 24h activated and rested mouse CD4 T cells stimulated with single and combinatorial cytokine conditions for 6 hours.

In [ ]:
counts = pd.read_csv('/data1/rudenska/EYW/git_projects/SIG14/processing_outs/count_matrix_umiDeDup_SIG14.csv', index_col=0)
feature_names = pd.read_csv('/data1/rudenska/EYW/git_projects/SIG14/processing_outs/featureNames_SIG14.csv', index_col=0)

# prepare counts matrix
counts_filtered = counts.loc[counts.index.isin(feature_names.index),:]
counts_filtered = counts_filtered.merge(feature_names, left_index=True, right_index=True)
counts_filtered = counts_filtered.set_index('gene')
counts_filtered.drop(columns=['category'], inplace=True)

# convert counts into anndata object and preprocess
rna = sc.AnnData(counts_filtered.T)
rna.var_names_make_unique()
sc.pp.normalize_total(rna)
sc.pp.log1p(rna)
rna.layers['log1p_norm'] = rna.X.copy()

# build out obs dataframe
obs_df = pd.DataFrame({"sample": counts_filtered.columns})
obs_df[['ligand1','ligand2','mouse','well','library','project']] = obs_df['sample'].str.split('_', expand=True)
obs_df = obs_df.assign(condition=lambda x: x['ligand1'] + '_' + x['ligand2'])
rna.obs = rna.obs.merge(obs_df, left_index=True, right_on='sample')

# average across treatment conditions
rna_pb = sc.get.aggregate(rna, by=['condition','ligand1','ligand2','mouse','well'], func='mean', layer='log1p_norm')
rna_pb.layers['log1p_norm'] = rna_pb.layers['mean'].copy()

/home/wange7/miniforge3/envs/scanpy_standard2/lib/python3.13/site-packages/anndata/_core/anndata.py:1798: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/wange7/miniforge3/envs/scanpy_standard2/lib/python3.13/functools.py:934: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


## SIG21 Import

Bulk RNA-seq of 72h activated and rested human CD4 T cells stimulated with single and combinatorial cytokine conditions for 6 hours.

In [ ]:
counts = pd.read_csv('/data1/rudenska/EYW/git_projects/SIG21/processing_outs/7DZBSH-expression-matrix_processed.csv', index_col=0)
feature_names = pd.read_csv('/data1/rudenska/EYW/git_projects/SIG21/processing_outs/7DZBSH-featureNames.csv', index_col=0)

# prepare counts matrix
counts_filtered = counts.merge(feature_names, left_index=True, right_on='gene_id')
counts_filtered = counts_filtered.set_index('gene_name')
counts_filtered.drop(columns=['gene_id','gene_biotype'], inplace=True)

# convert counts into anndata object and preprocess
rna = sc.AnnData(counts_filtered.T)
rna.var_names_make_unique()
sc.pp.normalize_total(rna)
sc.pp.log1p(rna)
rna.layers['log1p_norm'] = rna.X.copy()

# build out obs dataframe
obs_df = pd.DataFrame({"sample": counts_filtered.columns})
obs_df[['ligand1','ligand2','replicate']] = obs_df['sample'].str.split('_', expand=True)
obs_df = obs_df.assign(condition=lambda x: x['ligand1'] + '_' + x['ligand2'])
rna.obs = rna.obs.merge(obs_df, left_index=True, right_on='sample')

# average across treatment conditions
rna_pb = sc.get.aggregate(rna, by=['condition','ligand1','ligand2','replicate'], func='mean', layer='log1p_norm')
rna_pb.layers['log1p_norm'] = rna_pb.layers['mean'].copy()

# Define Model Functions

This function performs ridge regression parallelized across threads and returns activity scores (coefficients), chosen alpha values, and r2 values.

In [ ]:
def _process_single_condition(y_obs, X, alpha_range, n_perms=1000, zscore_coeffs=True):
    cv_model = RidgeCV(alphas=alpha_range, fit_intercept=True, cv=5).fit(X, y_obs)
    best_alpha, beta_obs, r2_obs = cv_model.alpha_, cv_model.coef_, cv_model.score(X, y_obs)

    rng = np.random.default_rng()
    Y_perm_matrix = np.array([rng.permutation(y_obs) for _ in range(n_perms)]).T

    perm_model = Ridge(alpha=best_alpha, fit_intercept=True, random_state=67).fit(X, Y_perm_matrix)
    beta_perms = perm_model.coef_ 

    with np.errstate(divide='ignore', invalid='ignore'):
        z = (beta_obs - np.mean(beta_perms, axis=0)) / np.std(beta_perms, axis=0) if zscore_coeffs else beta_obs
    
    return z, r2_obs, best_alpha

def calculate_ligand_activity_parallel(X_mat, Y_mat, alpha_range=np.logspace(-1, 3, 100), n_jobs=-1, verbose=0, zscore_coeffs=True):
    X = X_mat.values if isinstance(X_mat, pd.DataFrame) else X_mat
    Y = Y_mat.values if isinstance(Y_mat, pd.DataFrame) else Y_mat
    ligand_names = X_mat.columns if isinstance(X_mat, pd.DataFrame) else np.arange(X.shape[1])
    cond_names = Y_mat.columns if isinstance(Y_mat, pd.DataFrame) else np.arange(Y.shape[1])
    
    results = Parallel(n_jobs=n_jobs, verbose=verbose)(
        delayed(_process_single_condition)(Y[:, i], X, alpha_range, zscore_coeffs=zscore_coeffs) for i in range(len(cond_names))
    )

    zs, r2s, alphas = zip(*results)
    
    if verbose > 0:
        for name, a, r in zip(cond_names, alphas, r2s):
            print(f"Condition: {name} | Chosen Alpha: {a:.5f} | R2: {r:.4f}")

    return pd.DataFrame(dict(zip(cond_names, zs)), index=ligand_names), pd.Series(r2s, index=cond_names)


# Model Activity Scoring

Here, we will calculate activity score model outputs across different parameters.

In [ ]:
# import and scale explanatory matrix
ligand_scores = pd.read_csv("/data1/rudenska/EYW/git_projects/SIG13/analysis_outs/cytosig/SIG13_waggr_scores_explanatory_mat.csv", index_col=0)
ligands_df = ligand_scores.T
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(ligands_df), columns=ligands_df.columns, index=ligands_df.index)

## Calculate SIG14 sPCA scores

In [ ]:
# import spca component genes
spca_components = pd.read_csv("/data1/rudenska/EYW/git_projects/SIG13/analysis_outs/spca/zscore_degs_allLigands_0.1_alpha1.0_sPCA_loadings.csv")
lm_scored = pd.read_csv('/data1/rudenska/EYW/git_projects/SIG13/analysis_outs/spca/lm_scored_zscore_degs_allLigands_0.1_alpha1.0_sPCA_clean.csv')

# filter for good components
good_comps = lm_scored['component'].unique().tolist()
spca_components = spca_components[spca_components['spca_component'].isin(good_comps)]
# format for decoupler
net = spca_components.rename(columns={'gene':'target',
                                      'spca_component':'source',
                                      'loading':'weight'})
net_50_weighted = net.groupby('source', group_keys=False).apply(lambda x: x.nlargest(50, 'weight'))
net_50_unweighted = net.groupby('source', group_keys=False).apply(lambda x: x.nlargest(50, 'weight')).drop('weight', axis=1)
net_all_weighted = net.copy()
net_all_unweighted = net.drop('weight', axis=1)

# define genes used in scoring
comp_genes = net['target'].unique().tolist()

/tmp/ipykernel_2436577/2167409987.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  net_50_weighted = net.groupby('source', group_keys=False).apply(lambda x: x.nlargest(50, 'weight'))
/tmp/ipykernel_2436577/2167409987.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  net_50_unweighted = net.groupby('source', group_keys=False).apply(lambda x: x.nlargest(50, 'weight')).drop('weight', axis=1)


In [ ]:
def calc_spca_scores(adata, net, scoring_genes):
    # subset to genes used in scoring
    adata = adata.copy()
    adata_sub = adata[:, adata.var_names.isin(scoring_genes)].copy()
    adata_sub.X = adata_sub.layers['log1p_norm']
    sc.pp.scale(adata_sub)
    
    # calculate waggr with no iterations (no pvalues)
    dc.mt.waggr(adata_sub, net, tmin=5, times=0)
    
    # add waggr scores to full anndata
    adata.obsm['score_waggr'] = adata_sub.obsm['score_waggr'].copy()

    # add waggr scores to metadata
    score_df = adata_sub.obsm['score_waggr']
    score_df.columns = [f'waggr_{col}' for col in score_df.columns]
    adata.obs = pd.concat([adata_sub.obs, score_df], axis=1)

    return adata

In [ ]:
rna_scored_50_weighted = calc_spca_scores(rna_pb, net_50_weighted, comp_genes)
rna_scored_50_unweighted = calc_spca_scores(rna_pb, net_50_unweighted, comp_genes)
rna_scored_all_weighted = calc_spca_scores(rna_pb, net_all_weighted, comp_genes)
rna_scored_all_unweighted = calc_spca_scores(rna_pb, net_all_unweighted, comp_genes)

## Calculate SIG14 Activity Scores

In [ ]:
def score_ridge(adata_scored):
    # process y matrix
    y_df = (adata_scored.obs.assign(sample=lambda x: x['condition'].astype(str) + "_" + x['mouse'].astype(str) + "_" + x['well'].astype(str))
                   .groupby('sample')[[c for c in adata_scored.obs.columns if c.startswith('waggr_')]]
                   .mean()
                   .T)
    y_df.index = y_df.index.str.replace('waggr_', '')
    Y_scaled = pd.DataFrame(scaler.fit_transform(y_df), columns=y_df.columns, index=y_df.index)
    
    # calculate ligand activity scores
    activity_scores, r2_scores = calculate_ligand_activity_parallel(X_scaled, Y_scaled, verbose=False,
                                                                   alpha_range=np.logspace(-1, 3, 100))
    activity_scores = activity_scores.T.reset_index(names='sample')

    # calculate ligand activity scores
    activity_scores_coeff, r2_scores_coeff = calculate_ligand_activity_parallel(X_scaled, Y_scaled, verbose=False,
                                                                   alpha_range=np.logspace(-1, 3, 100),
                                                                   zscore_coeffs=False)
    activity_scores_coeff = activity_scores_coeff.T.reset_index(names='sample')
    
    return activity_scores, r2_scores, activity_scores_coeff, r2_scores_coeff

In [ ]:
# calculate activity scores
activity_zscore_ridge_50_unweighted, fit_zscore_ridge_50_unweighted, activity_coeff_ridge_50_unweighted, fit_coeff_ridge_50_unweighted = score_ridge(rna_scored_50_unweighted)
activity_zscore_ridge_50_weighted, fit_zscore_ridge_50_weighted, activity_coeff_ridge_50_weighted, fit_coeff_ridge_50_weighted = score_ridge(rna_scored_50_weighted)
activity_zscore_ridge_all_unweighted, fit_zscore_ridge_all_unweighted, activity_coeff_ridge_all_unweighted, fit_coeff_ridge_all_unweighted = score_ridge(rna_scored_all_unweighted)
activity_zscore_ridge_all_weighted, fit_zscore_ridge_all_weighted, activity_coeff_ridge_all_weighted, fit_coeff_ridge_all_weighted = score_ridge(rna_scored_all_weighted)

# concatenate all activity scores
activity_list = [
    activity_zscore_ridge_50_unweighted,
    activity_zscore_ridge_50_weighted,
    activity_zscore_ridge_all_unweighted,
    activity_zscore_ridge_all_weighted,
    activity_coeff_ridge_50_unweighted,
    activity_coeff_ridge_50_weighted,
    activity_coeff_ridge_all_unweighted,
    activity_coeff_ridge_all_weighted]
r2_list = [
    fit_zscore_ridge_50_unweighted,
    fit_zscore_ridge_50_weighted,
    fit_zscore_ridge_all_unweighted,
    fit_zscore_ridge_all_weighted,
    fit_coeff_ridge_50_unweighted,
    fit_coeff_ridge_50_weighted,
    fit_coeff_ridge_all_unweighted,
    fit_coeff_ridge_all_weighted]
activity_combined = pd.concat([
    df.assign(model=name) for df, name in zip(
        activity_list,
        ['ridge_zscore_50_unweighted', 'ridge_zscore_50_weighted', 'ridge_zscore_all_unweighted', 'ridge_zscore_all_weighted',
         'ridge_coeff_50_unweighted', 'ridge_coeff_50_weighted', 'ridge_coeff_all_unweighted', 'ridge_coeff_all_weighted']
    )
], ignore_index=True)

# Concatenate all R2 scores
r2_combined = pd.concat([
    pd.DataFrame({'sample': r2.index, 'r2_score': r2.values, 'model': name})
    for r2, name in zip(
        r2_list,
        ['ridge_zscore_50_unweighted', 'ridge_zscore_50_weighted', 'ridge_zscore_all_unweighted', 'ridge_zscore_all_weighted',
         'ridge_coeff_50_unweighted', 'ridge_coeff_50_weighted', 'ridge_coeff_all_unweighted', 'ridge_coeff_all_weighted']
    )
], ignore_index=True)

In [ ]:
# Export scores
# activity_combined.to_csv('/data1/rudenska/EYW/git_projects/SIG13/analysis_outs/cytosig/model_optimization/activity_combined_SIG14.csv', index=False)
# r2_combined.to_csv('/data1/rudenska/EYW/git_projects/SIG13/analysis_outs/cytosig/model_optimization/r2_combined_SIG14.csv', index=False)

## Calculate SIG21 sPCA scores

In [20]:
spca_components = pd.read_csv("/data1/rudenska/EYW/git_projects/SIG13/analysis_outs/spca/zscore_degs_allLigands_0.1_alpha1.0_sPCA_loadings.csv")
lm_scored = pd.read_csv('/data1/rudenska/EYW/git_projects/SIG13/analysis_outs/spca/lm_scored_zscore_degs_allLigands_0.1_alpha1.0_sPCA_clean.csv')

In [21]:
# filter for good components
good_comps = lm_scored['component'].unique().tolist()
spca_components = spca_components[spca_components['spca_component'].isin(good_comps)]
# format for decoupler
net = spca_components.rename(columns={'gene':'target',
                                      'spca_component':'source',
                                      'loading':'weight'})
# convert to human genes
net = scc.convert_mouse_genes_to_human(net,'target')
net.drop('target', axis=1, inplace=True)
net.rename(columns={'human_gene':'target'}, inplace=True)
net.drop_duplicates(subset=['source', 'target'], inplace=True)

net_50_weighted = net.groupby('source', group_keys=False).apply(lambda x: x.nlargest(50, 'weight'))
net_50_unweighted = net.groupby('source', group_keys=False).apply(lambda x: x.nlargest(50, 'weight')).drop('weight', axis=1)
net_all_weighted = net.copy()
net_all_unweighted = net.drop('weight', axis=1)

# define genes used in scoring
comp_genes = net['target'].unique().tolist()

/tmp/ipykernel_3943580/1096132307.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  net_50_weighted = net.groupby('source', group_keys=False).apply(lambda x: x.nlargest(50, 'weight'))
/tmp/ipykernel_3943580/1096132307.py:15: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  net_50_unweighted = net.groupby('source', group_keys=False).apply(lambda x: x.nlargest(50, 'weight')).drop('weight', axis=1)


In [ ]:
def calc_spca_scores(adata, net, scoring_genes):
    # subset to genes used in scoring
    adata = adata.copy()
    adata_sub = adata[:, adata.var_names.isin(scoring_genes)].copy()
    adata_sub.X = adata_sub.layers['log1p_norm']
    sc.pp.scale(adata_sub)
    
    # calculate waggr with no iterations (no pvalues)
    dc.mt.waggr(adata_sub, net, tmin=5, times=0)
    
    # add waggr scores to full anndata
    adata.obsm['score_waggr'] = adata_sub.obsm['score_waggr'].copy()

    # add waggr scores to metadata
    score_df = adata_sub.obsm['score_waggr']
    score_df.columns = [f'waggr_{col}' for col in score_df.columns]
    adata.obs = pd.concat([adata_sub.obs, score_df], axis=1)

    return adata

In [ ]:
rna_scored_50_weighted = calc_spca_scores(rna_pb, net_50_weighted, comp_genes)
rna_scored_50_unweighted = calc_spca_scores(rna_pb, net_50_unweighted, comp_genes)
rna_scored_all_weighted = calc_spca_scores(rna_pb, net_all_weighted, comp_genes)
rna_scored_all_unweighted = calc_spca_scores(rna_pb, net_all_unweighted, comp_genes)

# Calculate SIG21 Activity Scores

In [ ]:
def score_ridge(adata_scored):
    # process y matrix
    y_df = (adata_scored.obs.assign(sample=lambda x: x['condition'].astype(str) + "_" + x['replicate'].astype(str))
                   .groupby('sample')[[c for c in adata_scored.obs.columns if c.startswith('waggr_')]]
                   .mean()
                   .T)
    y_df.index = y_df.index.str.replace('waggr_', '')
    Y_scaled = pd.DataFrame(scaler.fit_transform(y_df), columns=y_df.columns, index=y_df.index)
    
    # calculate ligand activity scores
    activity_scores, r2_scores = calculate_ligand_activity_parallel(X_scaled, Y_scaled, verbose=False,
                                                                   alpha_range=np.logspace(-1, 3, 100))
    activity_scores = activity_scores.T.reset_index(names='sample')

    # calculate ligand activity scores
    activity_scores_coeff, r2_scores_coeff = calculate_ligand_activity_parallel(X_scaled, Y_scaled, verbose=False,
                                                                   alpha_range=np.logspace(-1, 3, 100),
                                                                   zscore_coeffs=False)
    activity_scores_coeff = activity_scores_coeff.T.reset_index(names='sample')
    
    return activity_scores, r2_scores, activity_scores_coeff, r2_scores_coeff

In [ ]:
# calculate activity scores
activity_zscore_ridge_50_unweighted, fit_zscore_ridge_50_unweighted, activity_coeff_ridge_50_unweighted, fit_coeff_ridge_50_unweighted = score_ridge(rna_scored_50_unweighted)
activity_zscore_ridge_50_weighted, fit_zscore_ridge_50_weighted, activity_coeff_ridge_50_weighted, fit_coeff_ridge_50_weighted = score_ridge(rna_scored_50_weighted)
activity_zscore_ridge_all_unweighted, fit_zscore_ridge_all_unweighted, activity_coeff_ridge_all_unweighted, fit_coeff_ridge_all_unweighted = score_ridge(rna_scored_all_unweighted)
activity_zscore_ridge_all_weighted, fit_zscore_ridge_all_weighted, activity_coeff_ridge_all_weighted, fit_coeff_ridge_all_weighted = score_ridge(rna_scored_all_weighted)

# concatenate all activity scores
activity_list = [
    activity_zscore_ridge_50_unweighted,
    activity_zscore_ridge_50_weighted,
    activity_zscore_ridge_all_unweighted,
    activity_zscore_ridge_all_weighted,
    activity_coeff_ridge_50_unweighted,
    activity_coeff_ridge_50_weighted,
    activity_coeff_ridge_all_unweighted,
    activity_coeff_ridge_all_weighted]
r2_list = [
    fit_zscore_ridge_50_unweighted,
    fit_zscore_ridge_50_weighted,
    fit_zscore_ridge_all_unweighted,
    fit_zscore_ridge_all_weighted,
    fit_coeff_ridge_50_unweighted,
    fit_coeff_ridge_50_weighted,
    fit_coeff_ridge_all_unweighted,
    fit_coeff_ridge_all_weighted]
activity_combined = pd.concat([
    df.assign(model=name) for df, name in zip(
        activity_list,
        ['ridge_zscore_50_unweighted', 'ridge_zscore_50_weighted', 'ridge_zscore_all_unweighted', 'ridge_zscore_all_weighted',
         'ridge_coeff_50_unweighted', 'ridge_coeff_50_weighted', 'ridge_coeff_all_unweighted', 'ridge_coeff_all_weighted']
    )
], ignore_index=True)

# Concatenate all R2 scores
r2_combined = pd.concat([
    pd.DataFrame({'sample': r2.index, 'r2_score': r2.values, 'model': name})
    for r2, name in zip(
        r2_list,
        ['ridge_zscore_50_unweighted', 'ridge_zscore_50_weighted', 'ridge_zscore_all_unweighted', 'ridge_zscore_all_weighted',
         'ridge_coeff_50_unweighted', 'ridge_coeff_50_weighted', 'ridge_coeff_all_unweighted', 'ridge_coeff_all_weighted']
    )
], ignore_index=True)

In [ ]:
# Export
# activity_combined.to_csv('/data1/rudenska/EYW/git_projects/SIG13/analysis_outs/cytosig/model_optimization/activity_combined_SIG21.csv', index=False)
# r2_combined.to_csv('/data1/rudenska/EYW/git_projects/SIG13/analysis_outs/cytosig/model_optimization/r2_combined_SIG21.csv', index=False)